In [2]:
import nltk
import pandas as pd 
import numpy as np
import re
import ast
from tqdm import tqdm
from gensim.models import Word2Vec
from scipy.spatial.distance import cosine
import gc
import json
import pickle


In [3]:
df_new = pd.read_csv("/kaggle/input/parliament/speeches_new.csv")
df_old = pd.read_csv("/kaggle/input/old-parliament/speeches_old.csv")

/tmp/ipykernel_33/2297234945.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_new = pd.read_csv("/kaggle/input/parliament/speeches_clean.csv")


In [3]:
df_old.rename(columns={"sentence": "speechContent"}, inplace=True)
df_old.head()

,id,begin,end,speechContent,dateDay,dateMonth,dateYear,timestamp
0,49,0,37,11 Sitzung\n\nam Donnerstag den 3. März,3,3,1887,-2613945600000
1,61,38,43,1887.,3,3,1887,-2613945600000
2,73,43,53,\n\n\nSeite\n\n,3,3,1887,-2613945600000
3,85,53,111,Vorläufige Konstituirung des Reichstags 1........,3,3,1887,-2613945600000
4,97,112,116,33\n\n,3,3,1887,-2613945600000


In [4]:
tqdm.pandas()
def preprocess(df, length_threshold):  
    def remove_special_characters(text):
        # Define a regular expression pattern that matches all non-alphanumeric characters except for ä, ö, ü
        pattern = r'[^a-zA-Z0-9äöüÄÖÜ\s]'
        # Replace special characters with an empty string
        clean_text = re.sub(pattern, '', text)
        return clean_text

    # Clean text
    df['speechContent'] = df["speechContent"].astype(str)
    df["speechContent"].replace(to_replace=r"\.\.+", value=" ", regex=True, inplace=True)
    df["speechContent"].replace(to_replace=r"\-\-+", value=" ", regex=True, inplace=True)
    df["speechContent"].replace(to_replace=r"__+", value=" ", regex=True, inplace=True)
    df["speechContent"].replace(to_replace=r"\*\*+", value=" ", regex=True, inplace=True)
    df["speechContent"].replace(to_replace=r"\s+", value=" ", regex=True, inplace=True)
    df["speechContent"] = df["speechContent"].progress_apply(remove_special_characters)
    
    df['length'] = df["speechContent"].progress_apply(lambda x: len(x.split()))
    
    
    df = df[df['length'] > length_threshold]
    
    return df
    

df_old = preprocess(df_old, 10)
df_new = preprocess(df_new, 10)

/tmp/ipykernel_33/3975187826.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["speechContent"].replace(to_replace=r"\.\.+", value=" ", regex=True, inplace=True)
/tmp/ipykernel_33/3975187826.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col]

In [5]:
tqdm.pandas()
df_old['speechTokens'] = df_old["speechContent"].progress_apply(lambda x: x.split())
df_new['speechTokens'] = df_new["speechContent"].progress_apply(lambda x: x.split())
df_old.head()

100%|██████████| 346685/346685 [00:28<00:00, 12100.84it/s]


,id,begin,end,speechContent,dateDay,dateMonth,dateYear,timestamp,length,speechTokens
6,121,221,327,33 Ergebni der Verloosung der Wahlakten an die...,3,3,1887,-2613945600000,13,"[33, Ergebni, der, Verloosung, der, Wahlakten,..."
8,145,347,456,Diskussion zur Geschäftsordnung betreffend die...,3,3,1887,-2613945600000,13,"[Diskussion, zur, Geschäftsordnung, betreffend..."
10,169,496,586,4 6 88 Staatssekretär des Innern Staatsministe...,3,3,1887,-2613945600000,11,"[4, 6, 88, Staatssekretär, des, Innern, Staats..."
16,241,706,831,1 6 77 von Kardorff 1 1 7 88 Feststellung der ...,3,3,1887,-2613945600000,17,"[1, 6, 77, von, Kardorff, 1, 1, 7, 88, Festste..."
19,277,890,982,Die Sitzung wird um 11 Uhr 35 Minuten durch de...,3,3,1887,-2613945600000,16,"[Die, Sitzung, wird, um, 11, Uhr, 35, Minuten,..."


In [9]:
# Create a list of tokenized sentences
sentences_new = df_new['speechTokens'].tolist()
sentences_old = df_old['speechTokens'].tolist()
sentences = sentences_new + sentences_old

with open('training_corpus.pkl', 'wb') as f:
    pickle.dump(sentences, f)


In [4]:
print(type(sentences))   

# Define the Word2Vec model
model = Word2Vec(
    vector_size=300,
    window=10,
    min_count=1,
    workers=4,
)

print("Building Vocabulary")

# Build vocabulary
model.build_vocab(sentences)

print("Start Training")
# Training with progress bar for each epoch
epochs = 10
total_sentences = len(sentences)

for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')
    pbar = tqdm(total=total_sentences, desc=f"Epoch {epoch+1}", unit="sentence")
    
    # Shuffle sentences to ensure different batches in each epoch
    np.random.shuffle(sentences)
    
    # Train in batches to update the progress bar
    batch_size = 100000  # Define your batch size
    for i in range(0, total_sentences, batch_size):
        batch_sentences = sentences[i:i + batch_size]
        model.train(batch_sentences, total_examples=len(batch_sentences), epochs=1)
        pbar.update(len(batch_sentences))
    
    pbar.close()

# Save the model
model.save("word2vec.model")

# Example: Load the model and find similar words
#model = Word2Vec.load("word2vec_new.model")

<class 'list'>
Building Vocabulary
Start Training
Epoch 1/10


Epoch 1: 100%|██████████| 5391688/5391688 [07:07<00:00, 12605.67sentence/s]


Epoch 2/10


Epoch 2: 100%|██████████| 5391688/5391688 [07:02<00:00, 12762.48sentence/s]


Epoch 3/10


...

Epoch 8: 100%|██████████| 5391688/5391688 [07:00<00:00, 12806.98sentence/s]


Epoch 9/10


Epoch 9: 100%|██████████| 5391688/5391688 [07:01<00:00, 12789.36sentence/s]


Epoch 10/10


Epoch 10: 100%|██████████| 5391688/5391688 [07:01<00:00, 12804.12sentence/s]
